# Retrieval Augmented Generation (RAG) Using our Vector DB

In section I, we built a Vector DB to allow for retrieval of similar documents.  This direct followup will show how to use the Vector DB to enhance our prompts with additional context before we put it into a Large Language Model.  

The notebook follows as:

1. RAG Conceptually
   - Question-Answering using Large Language Models
   - Retrieval of Relevant Documents for a Query
   - Question-Answering using RAG for Document Context
2. Using built-in LangChain RAG prompts and Vectors

## 1. RAG Conceptually

Large Language Models have proven to be very good at general question and answering tasks.  However, a main limitation of many LLMs is that they are generally constrained to the data that they are initially trained on.  Without access to an external data source, LLMs cannot bring in new information, whether this is proprietary domain specific knowledge or just an update on an existing knowledge base.  Given that, how can we enable LLMs to be updated with new information while leveraging the powerful language properties?

One solution to this Retrieval Augumented Generation (RAG).  In RAG, we leverage the fact that LLMs can be prompted with additional context data to add additional relevant context to a given query before we pass it into the model.  The old pipeline would be:

```
Query ------> LLM
```

which with RAG will be updated to

```
Query ------> Retrieve Relevant Documents ------> Augmented Query ------> LLM
```

We will retrieve relevant documents using the knowledge base we built with the Vector DB.

### Question-Answering using Large Language Models

We start by looking at a question answering system that simply asks the LLM a question.  In this case, if the model doesn't already know the answer, then there's not much way to inject that knowledge into the model.  Some models may immediately identify that there's not enough context while other models may go off rails and hallucinate.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
# THE FIRST TIME YOU RUN THIS, IT MIGHT TAKE A WHILE

model_path_or_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_path_or_id)
model = AutoModelForCausalLM.from_pretrained(
    model_path_or_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    bnb_4bit_compute_dtype=torch.float16,
    use_flash_attention_2=True,
    load_in_4bit=True
)

def generate(prompt):
    """Convenience function for generating model output"""
    # Tokenize the input
    input_ids = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True).input_ids.cuda()
    
    # Generate new tokens based on the prompt, up to max_new_tokens
    # Sample aacording to the parameter
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=input_ids, 
            max_new_tokens=100, 
            do_sample=True, 
            top_p=0.9,
            temperature=0.9,
            use_cache=True
        )
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Let's ask it a very general question because the ChatGPT has been trained on a huge amount of data and providing any specifics in the question will likely result in a correct answer.  In this situation, the model can't possibly ground itself because it doesn't know the context - yet it will still answer with something that it has.

In [7]:
# Prepare the input for for tokenization, attach any prompt that should be needed
PROMPT_TEMPLATE = """
    Question: {query}

    Answer: 
"""

query = "What's the efficacy of NeuroGlyde?"
prompt = PROMPT_TEMPLATE.format(query = query)

res = generate(prompt)

print(f"Prompt:\n{prompt}\n")
print(f"Generated Response:\n{res}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt:

    Question: What's the efficacy of NeuroGlyde?

    Answer: 


Generated Response:
      NeuroGlyde is a product that claims to improve memory and cognitive function. It is a dietary supplement that contains a blend of herbs, vitamins, and minerals that are believed to support brain health. However, there is limited research on the effectiveness of NeuroGlyde for memory and cognitive function. Some studies have shown that certain ingredients in NeuroGlyde, such as Bacillus coagulans, may improve cognitive function in healthy adults


It doesn't know the context, so let's provide it the context.  Which context should we provide?  The context will be retrieved from our vector databse.

We will retrieve the relevant documents to this question, inject it into the prompt, and send that to the model instead.

### Retrieval of Relevant Documents for a Query

We'll briefly revisit our code to retrieve documents from our previous example.  This Vector DB has already been populated with a set of documents.

In [2]:
from typing import List, Dict
from langchain.vectorstores.pgvector import PGVector

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [3]:
# The connection to the database
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver= "psycopg2",
    host = "localhost",
    port = "5432",
    database = "postgres",
    user= "username",
    password="password"
)

# The embedding function that will be used to store into the database
embedding_function = SentenceTransformerEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs = {'device': 'cuda'},
    encode_kwargs = {'normalize_embeddings': True}
)

# Creates the database connection to our existing DB
db = PGVector(
    connection_string = CONNECTION_STRING,
    collection_name = "embeddings",
    embedding_function = embedding_function,
    pre_delete_collection = True, # uncomment this to delete existing database first
    
)

#### (restart database)

In [4]:
from typing import List, Dict

In [5]:
from langchain_core.documents import Document

import glob
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector

import sqlalchemy

In [6]:
def chunk_document(doc_path: str) -> List[Document]:
    """Chunk a document into smaller langchain Documents for embedding.

    :param doc_path: path to document
    :type doc_path: str
    :return: List of Document chunks
    :rtype: List[Document]
    """
    loader = PyPDFLoader(doc_path)
    documents = loader.load()

    # split document based on the `\n\n` character, quite unintuitive
    # https://stackoverflow.com/questions/76633836/what-does-langchain-charactertextsplitters-chunk-size-param-even-do
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    
    return text_splitter.split_documents(documents)

# load the document and split it into chunks
doc_chunks = []
for doc in glob.glob("../../msl-data/*.pdf"):
    doc_chunks += chunk_document(doc)

In [7]:
len(doc_chunks)

0

In [ ]:
db = PGVector.from_documents(
    doc_chunks[:5],
    connection_string = CONNECTION_STRING,
    collection_name = "embeddings",
    embedding = embedding_function,
    pre_delete_collection = True, # uncomment this to delete existing database first
)

In [14]:
# query it, note that the score here is a distance metric (lower is more related)
# query = "What's the efficacy of NeuroGlyde?"
query = "What's the efficacy of Numpy?"
docs_with_scores = db.similarity_search_with_score(query, k = 1)

# print results
for doc, score in docs_with_scores:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.26005329906781316
NumPy (Numerical Python) is an open source Python library that's used in almost every field of science and engineering. It's the universal standard for working with numerical data in Python, and it's at the core of the scientific Python and PyData ecosystems.
--------------------------------------------------------------------------------


In [11]:
docs_with_scores

[(Document(page_content="NumPy (Numerical Python) is an open source Python library that's used in almost every field of science and engineering. It's the universal standard for working with numerical data in Python, and it's at the core of the scientific Python and PyData ecosystems.", metadata={'source': 'sample_docs/sample_docs-Copy4.txt'}),
  0.6104906135505066)]

When we query, we get the most relevant document for this query.  Let's create a new prompt that can take this new context. 

### Question-Answering using RAG for Document Context

In [ ]:
# Prepare the input for for tokenization, attach any prompt that should be needed
RAG_PROMPT_TEMPLATE = """
Answer the question using only this context:

Context: {context}

Question: {query}

Answer: 
"""

query = "What's the efficacy of NeuroGlyde?"
docs_with_scores = db.similarity_search_with_score(query, k = 1)
context_prompt = RAG_PROMPT_TEMPLATE.format(
    context = docs_with_scores[0][0].page_content,
    query = query
)

res = generate(context_prompt)

print(f"Prompt:\n{context_prompt}\n")
print(f"Generated Response:\n{res}")

That's it! That's the general concept of Retrieval Augmented Generation.

## Using built in LangChain RAG chains instead

LangChain contains many built-in methods that have connectivity to Vector Databases and LLMs.  In the example above, we built a custom prompt template and manually retrieved the document, then put it into the chain.  While pretty simple, with LangChain, this can all be pipelined together and more can be done, such as retrieving meta-data and sources.

In [ ]:
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableParallel
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

# Turn our db into a retriever
retriever = db.as_retriever(search_kwargs = {'k' : 2})

# Turn our model into an LLM
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=pipe)

prompt_template = PromptTemplate.from_template("""
Answer the question using only this context:

Context: {context}

Question: {question}

Answer: 
""")                                    

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build a chain with multiple documents for RAG
rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

# 2-step chain, first retrieve documents
# Then take those documents and store relevant infomration in `document_sources`
# Pass the prompt into the document chain
rag_chain_with_source = RunnableParallel({
    "documents": retriever, 
     "question": RunnablePassthrough()
}) | {
    "sources": lambda input: [(doc.page_content, doc.metadata) for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

In [ ]:
res = rag_chain_with_source.invoke("What's the efficacy of Pentatryponal?")

In [ ]:
print(res['answer'])

In [ ]:
res['sources']